# Identify tags in airline database

## Recurrent neural network

    - Improving the model of: 01_identify_tags_in_airline_database_LSTM adding a inverse RNN and combine it.  


In [1]:
from __future__ import print_function

import os 
import numpy as np 

import tensorflow as tf 
print(tf.__version__)


0.11.0rc2


## Read data and create training datasets
    - Previous code sumarized

In [2]:
# Read data
import pickle

atis_file = '/home/jorge/data/training/text/atis/atis.pkl'
with open(atis_file,'rb') as f:
    train, test, dicts = pickle.load(f)

#Dictionaries and train test partition
w2idx, ne2idx, labels2idx = dicts[b'words2idx'], dicts[b'tables2idx'], dicts[b'labels2idx']
    
idx2w  = dict((v,k) for k,v in w2idx.items())
idx2la = dict((v,k) for k,v in labels2idx.items())

train_x, _, train_label = train
test_x,  _,  test_label  = test


# Max value of word coding to assign the ID_PAD
ID_PAD = np.max([np.max(tx) for tx in train_x]) + 1
print('ID_PAD: ', ID_PAD)

def context(l, size=3):
    l = list(l)
    lpadded = size // 2 * [ID_PAD] + l + size // 2 * [ID_PAD]
    out = [lpadded[i:(i + size)] for i in range(len(l))]
    return out


# Create train and test X y.
X_trn=[]
for s in train_x:
    X_trn += context(s,size=10)
X_trn = np.array(X_trn)

X_tst=[]
for s in test_x:
    X_tst += context(s,size=10)
X_tst = np.array(X_tst)

y_trn=[]
for s in train_label:
    y_trn += list(s)
y_trn = np.array(y_trn)

y_tst=[]
for s in test_label:
    y_tst += list(s)
y_tst = np.array(y_tst)


ID_PAD:  572


# Model

## Architecture
    - tf.nn.embedding_lookup
    - tf.nn.dynamic_rnn layer
    - Dense layer: tf.nn.relu(tf.matmul(x, W) + b)
    
## Features
    - Dropout
    - Saver

In [4]:
#General parameters
LOG_DIR = '/tmp/tensorboard/airline/lstm/'

# data attributes
input_seq_length = X_trn.shape[1]
input_vocabulary_size = len(set(idx2w)) + 1
output_length = 127

#Model parameters
embedding_size = 64
num_hidden_lstm = 128


In [ ]:
cell = tf.nn.rnn_cell.MultiRNNCell([single_cell] * num_layers)

In [13]:
from tensorflow.contrib.tensorboard.plugins import projector

# Define the tensorflow graph
graph = tf.Graph()
with graph.as_default():
    # graph definition
    # Inputs
    with tf.name_scope('Inputs') as scope:
        x = tf.placeholder(tf.int32, shape=[None, input_seq_length], name='x')
        y = tf.placeholder(tf.int64, shape=[None], name='y')

    with tf.name_scope('Embeddings') as scope:
        W_embedding = tf.Variable(tf.random_uniform([input_vocabulary_size, embedding_size], -1.0, 1.0) ,name="W")
        embedding_layer = tf.nn.embedding_lookup(W_embedding, x)
        print('embedding_layer: ', embedding_layer)
        

    with tf.name_scope('RNN') as scope:
        # Dropout parameter
        keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        
        # Forward RNN cell
        cell_fw = tf.nn.rnn_cell.LSTMCell(num_hidden_lstm, 
                                          initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=123))
        cell_fw = tf.nn.rnn_cell.DropoutWrapper(cell_fw, output_keep_prob=keep_prob)
        
        # Backward RNN cell
        cell_bw = tf.nn.rnn_cell.LSTMCell(num_hidden_lstm, 
                                          initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=113))
        cell_bw = tf.nn.rnn_cell.DropoutWrapper(cell_bw, output_keep_prob=keep_prob)
        
        # Convert tensor in a list of tensors
        embedding_layer_split = tf.split(1, input_seq_length, embedding_layer)
        
        # Remove dimensions of shape 1
        embedding_layer_split = [tf.reshape(l, [-1, embedding_size]) for l in embedding_layer_split]
        
        # Bidirectional RR: output: list of concatenate outputs of the fw and bw networks.
        lstm_outputs, _, _ = tf.nn.bidirectional_rnn(cell_fw, cell_bw,
                                                                embedding_layer_split, dtype=tf.float32)
        print('lstm_outputs: ', lstm_outputs)
        
        # Concat the first and the last outputs of the network.
        final_lstm_outputs = tf.concat(1, [lstm_outputs[0], lstm_outputs[-1]])
        print('final_lstm_outputs: ', final_lstm_outputs)
        
    
    #Dense layer form RNN outs to prediction
    with tf.name_scope('Dense') as scope:
        W_dense = tf.Variable(tf.truncated_normal([num_hidden_lstm*2*2, output_length], stddev=0.1), name='W_dense')
        b_dense = tf.Variable(tf.constant(0.1, shape=[output_length]), name='b_dense')
        dense_output = tf.nn.relu(tf.matmul(final_lstm_outputs, W_dense) + b_dense)
        print('dense_output: ', dense_output)

        
    #Prediction
    y_pred = tf.nn.softmax(dense_output, name='y_pred')

    # Loss function
    with tf.name_scope("xent") as scope:
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(dense_output, y, name='cross_entropy')
        ce_summary = tf.scalar_summary("cross_entropy", tf.reduce_mean(cross_entropy))

    #Optimizer
    with tf.name_scope("train") as scope:
        optimizer = tf.train.AdamOptimizer(0.001)
        train_op = optimizer.minimize(cross_entropy, name='train_op')


    #Accuracy
    with tf.name_scope("test") as scope:
        correct_prediction = tf.equal(tf.argmax(dense_output,1), y)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
        accuracy_summary = tf.scalar_summary("accuracy", accuracy)

    # Merge all the summaries and write them out to /tmp/mnist_logs
    with tf.name_scope('summaries') as scope:
        merged = tf.merge_all_summaries()
        
        
    # Create a saver and save weigths.
    saver = tf.train.Saver()


embedding_layer:  Output("Embeddings/embedding_lookup:0", shape=(?, 10, 64), dtype=float32)
embedding_layer_split [<tf.Output 'RNN/split:0' shape=(?, 1, 64) dtype=float32>, <tf.Output 'RNN/split:1' shape=(?, 1, 64) dtype=float32>, <tf.Output 'RNN/split:2' shape=(?, 1, 64) dtype=float32>, <tf.Output 'RNN/split:3' shape=(?, 1, 64) dtype=float32>, <tf.Output 'RNN/split:4' shape=(?, 1, 64) dtype=float32>, <tf.Output 'RNN/split:5' shape=(?, 1, 64) dtype=float32>, <tf.Output 'RNN/split:6' shape=(?, 1, 64) dtype=float32>, <tf.Output 'RNN/split:7' shape=(?, 1, 64) dtype=float32>, <tf.Output 'RNN/split:8' shape=(?, 1, 64) dtype=float32>, <tf.Output 'RNN/split:9' shape=(?, 1, 64) dtype=float32>]
embedding_layer_split [<tf.Output 'RNN/Reshape:0' shape=(?, 64) dtype=float32>, <tf.Output 'RNN/Reshape_1:0' shape=(?, 64) dtype=float32>, <tf.Output 'RNN/Reshape_2:0' shape=(?, 64) dtype=float32>, <tf.Output 'RNN/Reshape_3:0' shape=(?, 64) dtype=float32>, <tf.Output 'RNN/Reshape_4:0' shape=(?, 64) dtype

In [14]:
#batch generator
def batch_generator(x, y, batch_size=128):
    for i in range(0, x.shape[0]-batch_size, batch_size):
        x_batch = x[i:i+batch_size,:]
        y_batch = y[i:i+batch_size]
        yield x_batch, y_batch
    
seq = batch_generator(X_trn, y_trn, batch_size=20)
print(next(seq))

(array([[572, 572, 572, 572, 572, 232, 542, 502, 196, 208],
       [572, 572, 572, 572, 232, 542, 502, 196, 208,  77],
       [572, 572, 572, 232, 542, 502, 196, 208,  77,  62],
       [572, 572, 232, 542, 502, 196, 208,  77,  62,  10],
       [572, 232, 542, 502, 196, 208,  77,  62,  10,  35],
       [232, 542, 502, 196, 208,  77,  62,  10,  35,  40],
       [542, 502, 196, 208,  77,  62,  10,  35,  40,  58],
       [502, 196, 208,  77,  62,  10,  35,  40,  58, 234],
       [196, 208,  77,  62,  10,  35,  40,  58, 234, 137],
       [208,  77,  62,  10,  35,  40,  58, 234, 137,  62],
       [ 77,  62,  10,  35,  40,  58, 234, 137,  62,  11],
       [ 62,  10,  35,  40,  58, 234, 137,  62,  11, 234],
       [ 10,  35,  40,  58, 234, 137,  62,  11, 234, 481],
       [ 35,  40,  58, 234, 137,  62,  11, 234, 481, 321],
       [ 40,  58, 234, 137,  62,  11, 234, 481, 321, 572],
       [ 58, 234, 137,  62,  11, 234, 481, 321, 572, 572],
       [234, 137,  62,  11, 234, 481, 321, 572, 572, 57

In [16]:
# Execute the graph to train a network
batch_size = 32
nEpochs = 30

gpu_options = tf.GPUOptions(allow_growth = True)
with tf.Session(graph=graph, config=tf.ConfigProto(gpu_options=gpu_options)) as session:
    
    #Create sumaries writers
    summaries_dir = '/tmp/tensorboard/airline/blstm'
    train_writer = tf.train.SummaryWriter(summaries_dir + '/train', session.graph, flush_secs=2)
    test_writer = tf.train.SummaryWriter(summaries_dir + '/test', flush_secs=2)

    print('Initializing')
    print('Epoch - Loss(trn) -  Acc(trn)   -   Loss(tst) -   Acc(tst)')
    tf.global_variables_initializer().run()
    for epoch in range(nEpochs):
        ce_c=[]
        acc_c=[]
        ce_c_tst=[]
        acc_c_tst=[]
        
        batch_list = batch_generator(X_trn, y_trn, batch_size=batch_size)
        for i, batch in enumerate(batch_list):
            feedDict = {x: batch[0], y: batch[1], keep_prob: 0.5} 
            _, ce, acc = session.run([train_op, cross_entropy, accuracy], feed_dict=feedDict)
            ce_c += [ce]
            acc_c += [acc]
        # Sumaries train    
        summary_str_trn = session.run(merged, feedDict)
        train_writer.add_summary(summary_str_trn, epoch)            
            
        batch_list_tst = batch_generator(X_tst, y_tst, batch_size=batch_size)
        for x_batch, y_batch in batch_list_tst:
            feedDict = {x: x_batch, y: y_batch, keep_prob: 1} 
            ce_tst, acc_tst = session.run([cross_entropy, accuracy], feed_dict=feedDict)
            ce_c_tst += [ce_tst]
            acc_c_tst += [acc_tst]
        # Sumaries test    
        summary_str_tst = session.run(merged, feedDict)
        test_writer.add_summary(summary_str_tst, epoch)            
        
        saver.save(session, os.path.join(LOG_DIR, "model.ckpt"), epoch)

        print(epoch, np.mean(ce_c), np.mean(acc_c), np.mean(ce_c_tst), np.mean(acc_c_tst), sep='   -   ')

Initializing
Epoch - Loss(trn) -  Acc(trn)   -   Loss(tst) -   Acc(tst)
0   -   1.08675   -   0.7792   -   0.765317   -   0.855292
1   -   0.504932   -   0.902326   -   0.478807   -   0.913981
2   -   0.359555   -   0.932551   -   0.425067   -   0.924869
3   -   0.300212   -   0.94282   -   0.356184   -   0.936629
4   -   0.252965   -   0.951057   -   0.358319   -   0.937391
5   -   0.238466   -   0.953567   -   0.363465   -   0.938262
6   -   0.229458   -   0.954999   -   0.359496   -   0.939678
7   -   0.225761   -   0.955175   -   0.359718   -   0.940658
8   -   0.219023   -   0.95643   -   0.358461   -   0.939569
9   -   0.217709   -   0.956607   -   0.362791   -   0.94142
10   -   0.21533   -   0.956713   -   0.355786   -   0.943706
11   -   0.213216   -   0.957067   -   0.360127   -   0.944033
12   -   0.211042   -   0.957721   -   0.3654   -   0.943598
13   -   0.209642   -   0.957738   -   0.358799   -   0.94436
14   -   0.211111   -   0.957402   -   0.376811   -   0.942291
15 